In [116]:
import pandas as pd
from gluonts.dataset.pandas import PandasDataset

df = pd.read_csv('Thesis Tracking Data 20230919.csv')
df = df[['Timestamp', 'Latitude', 'Longitude']]
# df = df.rename(columns={'Timestamp': 'timestamp'})
# df['timestamp'] = pd.to_datetime(df['timestamp'])

df = df.astype({'Timestamp': 'string'})
df['Timestamp'] = df['Timestamp'].str.replace(' \+0000', '')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# df.set_index('timestamp')

gap_fill_df = pd.DataFrame({
    'Timestamp': pd.date_range(df['Timestamp'].min(), df['Timestamp'].max(), freq='1s')
})

df = pd.merge(df, gap_fill_df, on='Timestamp', how='outer')
df = df.sort_values(by=['Timestamp'])
df['Latitude'] = df['Latitude'].interpolate()
df['Longitude'] = df['Longitude'].interpolate()

df['Item_ID'] = '2'
df.loc[0:2200, 'Item_ID'] = '1'

# lat_ds = PandasDataset.from_long_dataframe(df, target='Latitude', item_id='Item_ID', timestamp='Timestamp', freq='S')
#long_ds = PandasDataset.from_long_dataframe(df, target='Longitude', item_id='Item_ID', timestamp='Timestamp', freq='S')

# lat_ds

lat_df = df[['Timestamp', 'Latitude', 'Item_ID']]
long_df = df[['Timestamp', 'Longitude', 'Item_ID']]

/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_49346/1593892640.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Timestamp'] = df['Timestamp'].str.replace(' \+0000', '')


,Timestamp,Latitude,Item_ID
0,2023-09-19 06:14:58,14.637844,1
1,2023-09-19 06:14:59,14.637848,1
2,2023-09-19 06:15:00,14.637866,1
3,2023-09-19 06:15:01,14.637893,1
4,2023-09-19 06:15:02,14.637924,1
...,...,...,...
2255,2023-09-19 06:56:17,14.640038,2
2256,2023-09-19 06:56:18,14.640041,2
2485,2023-09-19 06:56:19,14.640044,2
2257,2023-09-19 06:56:20,14.640047,2


In [110]:
from gluonts.itertools import Map

class ProcessStartField():
    ts_id = 0
    
    def __call__(self, data):
        data["start"] = data["start"].to_timestamp()
        data["feat_static_cat"] = [self.ts_id]
        self.ts_id += 1
        
        return data

process_start = ProcessStartField()

# lat_list_ds = list(Map(process_start, lat_ds))
# long_list_ds = list(Map(process_start, long_ds))

# lat_list_ds

AssertionError: Dataframe index is not uniformly spaced. If your dataframe contains data from multiple series in the same column ("long" format), consider constructing the dataset with `PandasDataset.from_long_dataframe` instead.

In [ ]:
# Formatting data for work with Features

lat_df['target'] = [None if x == 15 else df['Latitude'].iloc[:x].tolist() for x in range(min(len(df), 15))]
long_df['target'] = [None if x == 15 else df['Longitude'].iloc[:x].tolist() for x in range(min(len(df), 15))]

lat_df.dropna()
long_df.dropna()

In [ ]:
# Logic behind this: each row will be used as test data, whose validation data is 
# found in the succeeding row
# It already incorporates one loop around Line A as a starting point

lat_df['target'] = [None if x < 2250 else df['Latitude'].iloc[2200:x].tolist() for x in range(len(df))]
long_df['target'] = [None if x < 2250 else df['Longitude'].iloc[2200:x].tolist() for x in range(len(df))]

lat_df.dropna()
long_df.dropna()

# To ensure that the prediction length is 10 gps coordinates long, we take a sample every 10 seconds
lat_df = lat_df.iloc[::10, :]
long_df = long_df.iloc[::10, :]


In [119]:
from datasets import Dataset, Features, Value, Sequence

features = Features(
    {
        'Timestamp': Value('timestamp[s]'),
        'target': Sequence(Value('float64')),
        'feat_static_cat': Sequence(Value('uint64')),
        'item_id': Value('string')
    }
)

lat_ds = Dataset.from_pandas(lat_df)


{'Timestamp': Timestamp('2023-09-19 06:14:58'),
 'Latitude': 14.637844495160426,
 'Item_ID': '1',
 '__index_level_0__': 0}

In [120]:
!pip install -q transformers datasets evaluate accelerate gluonts ujson

In [121]:
# Update the start period to a pandas period
from functools import partial

def convert_to_pandas_period(date, freq):
    return pd.Period(date, freq)

def transofrm_start_field(batch, freq):
    batch['start'] = [convert_to_pandas_period(date, freq) for date in batch["start"]]

"""


train_dataset.set_transform(partial(transform_start_field, freq=freq))
test_dataset.set_transform(partial(transform_start_field, freq=freq))
"""

'\n\n\ntrain_dataset.set_transform(partial(transform_start_field, freq=freq))\ntest_dataset.set_transform(partial(transform_start_field, freq=freq))\n'

In [123]:
from gluonts.time_feature import get_lags_for_frequency, time_features_from_frequency_str

freq='10s'
lags_seq = get_lags_for_frequency(freq)
time_features = time_features_from_frequency_str(freq)



In [ ]:
# Defining the transformer model
prediction_length = 10

from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction

config = TimeSeriesTransformerConfig(
    prediction_lnegth=prediction_length,
    context_length=2000,
    lags_sequence=lags_seq,
    num_time_features=len(time_features),
    num_static_categorical_features=1,
    cardinality=len(train_dataset),
    embedding_dimension=[1],

    encoder_layers=4,
    decoder_layers=4,
    d_model=32
)

model = TimeSeriesTransformerForPrediction(config)

In [124]:
from gluonts.time_feature import (
    time_features_from_frequency_str,
    TimeFeature,
    get_lags_for_frequency,
)
from gluonts.dataset.field_names import FieldName
from gluonts.transform import (
    AddAgeFeature,
    AddObservedValuesIndicator,
    AddTimeFeatures,
    AsNumpyArray,
    Chain,
    ExpectedNumInstanceSampler,
    InstanceSplitter,
    RemoveFields,
    SelectFields,
    SetField,
    TestSplitSampler,
    Transformation,
    ValidationSplitSampler,
    VstackFeatures,
    RenameFields,
)

In [ ]:
from transformers import PretrainedConfig


def create_transformation(freq: str, config: PretrainedConfig) -> Transformation:
    remove_field_names = []
    if config.num_static_real_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_REAL)
    if config.num_dynamic_real_features == 0:
        remove_field_names.append(FieldName.FEAT_DYNAMIC_REAL)
    if config.num_static_categorical_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_CAT)

    # a bit like torchvision.transforms.Compose
    return Chain(
        # step 1: remove static/dynamic fields if not specified
        [RemoveFields(field_names=remove_field_names)]
        # step 2: convert the data to NumPy (potentially not needed)
        + (
            [
                AsNumpyArray(
                    field=FieldName.FEAT_STATIC_CAT,
                    expected_ndim=1,
                    dtype=int,
                )
            ]
            if config.num_static_categorical_features > 0
            else []
        )
        + (
            [
                AsNumpyArray(
                    field=FieldName.FEAT_STATIC_REAL,
                    expected_ndim=1,
                )
            ]
            if config.num_static_real_features > 0
            else []
        )
        + [
            AsNumpyArray(
                field=FieldName.TARGET,
                # we expect an extra dim for the multivariate case:
                expected_ndim=1, ## cs: from expected_ndim=1 if config.input_size == 1 else 2,
            ),
            # step 3: handle the NaN's by filling in the target with zero
            # and return the mask (which is in the observed values)
            # true for observed values, false for nan's
            # the decoder uses this mask (no loss is incurred for unobserved values)
            # see loss_weights inside the xxxForPrediction model
            AddObservedValuesIndicator(
                target_field=FieldName.TARGET,
                output_field=FieldName.OBSERVED_VALUES,
            ),
            # step 4: add temporal features based on freq of the dataset
            # month of year in the case when freq="M"
            # these serve as positional encodings
            AddTimeFeatures(
                start_field=FieldName.START,
                target_field=FieldName.TARGET,
                output_field=FieldName.FEAT_TIME,
                time_features=time_features_from_frequency_str(freq),
                pred_length=config.prediction_length,
            ),
#             # step 5: add another temporal feature (just a single number)
#             # tells the model where in the life the value of the time series is
#             # sort of running counter
#             AddAgeFeature(
#                 target_field=FieldName.TARGET,
#                 output_field=FieldName.FEAT_AGE,
#                 pred_length=config.prediction_length,
#                 log_scale=True,
#             ),
            # step 6: vertically stack all the temporal features into the key FEAT_TIME
            VstackFeatures(
                output_field=FieldName.FEAT_TIME,
                input_fields=[FieldName.FEAT_TIME]  ## cs: from "[FieldName.FEAT_TIME, FieldName.FEAT_AGE]"
                + (
                    [FieldName.FEAT_DYNAMIC_REAL]
                    if config.num_dynamic_real_features > 0
                    else []
                ),
            ),
            # step 7: rename to match HuggingFace names
            RenameFields(
                mapping={
                    FieldName.FEAT_STATIC_CAT: "static_categorical_features",
                    FieldName.FEAT_STATIC_REAL: "static_real_features",
                    FieldName.FEAT_TIME: "time_features",
                    FieldName.TARGET: "values",
                    FieldName.OBSERVED_VALUES: "observed_mask",
                }
            ),
        ]
    )

# print(len(FieldName.FEAT_TIME),len(FieldName.FEAT_TIME), len(FieldName.FEAT_DYNAMIC_REAL))
for i in [FieldName.FEAT_STATIC_CAT, FieldName.FEAT_STATIC_REAL, FieldName.FEAT_TIME, FieldName.TARGET, FieldName.OBSERVED_VALUES]:
    print(i, len(i))

# INSTANCE SPLITTER

In [ ]:
from gluonts.transform.sampler import InstanceSampler
from typing import Optional


def create_instance_splitter(
    config: PretrainedConfig,
    mode: str,
    train_sampler: Optional[InstanceSampler] = None,
    validation_sampler: Optional[InstanceSampler] = None,
) -> Transformation:
    assert mode in ["train", "validation", "test"]

    instance_sampler = {
        "train": train_sampler
        or ExpectedNumInstanceSampler(
            num_instances=1.0, min_future=config.prediction_length
        ),
        "validation": validation_sampler
        or ValidationSplitSampler(min_future=config.prediction_length),
        "test": TestSplitSampler(),
    }[mode]

    return InstanceSplitter(
        target_field="values",
        is_pad_field=FieldName.IS_PAD,
        start_field=FieldName.START,
        forecast_start_field=FieldName.FORECAST_START,
        instance_sampler=instance_sampler,
        past_length=config.context_length + max(config.lags_sequence),
        future_length=config.prediction_length,
        time_series_fields=["time_features", "observed_mask"],
    )